In [1]:
%matplotlib widget

In [2]:
import numpy as np
from scipy import signal
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import ppscore

plt.style.use('ggplot')

from PfyMU.gait.train_classifier.core import load_datasets

In [3]:
from PfyMU.features import *

In [4]:
def test_feature_params(x, y, Feature, param_sets):
    FB = Bank(window_length=None, window_step=None)
    
    feat_spec = []
    for ps in param_sets:
        FB + Feature(**ps)
        feat_spec.append('_' + '_'.join([str(i) for i in ps.values()]))
    
    xf, fnames = FB.compute(x, fs=50.0, windowed=True, columns=[''])
    feat_names = [i[1:] + j for i, j in zip(fnames, feat_spec)]
    
    feats = pd.DataFrame(index=range(xf.shape[0]), columns=['Label']+feat_names, dtype='float')
    feats['Label'] = Y
    feats.iloc[:, 1:] = xf
    
    nrows = feats.shape[1] // 4 + (1 if feats.shape[1] % 4 > 0 else 0)
    f, ax = plt.subplots(nrows=nrows, ncols=4, figsize=(10, 2.5 * nrows), sharex=True)

    j, k = 0, 0
    for i, ft in enumerate(feat_names):
        if k > 3:
            j += 1
            k  = 0
        sns.violinplot(x='Label', y=ft, data=feats, ax=ax[j, k])
        ax[j, k].set_ylabel(None)
        ax[j, k].set_title(ft, fontsize=8)
        ax[j, k].set_xlabel(None)

        k += 1

    for k in range(4):
        ax[-1, k].set_xticklabels(['NG', 'G'])

    f.tight_layout()
    
    # PPScore plot
    df_predictors = ppscore.predictors(feats, 'Label', output='df')
    plt.figure(figsize=(10, 5))
    ax = sns.barplot(data=df_predictors, x="x", y="ppscore")
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")
    plt.tight_layout()

In [5]:
steps = {
    'jumping-rope': 0.15,
    'stairs-descending': 0.1,
    'stairs-ascending': 0.1,
    'jumping': 0.15,
    'lying': 0.15,
    'elevator-ascending': 0.15,
    'elevator-descending': 0.15,
    'running': 0.075,
    'sweeping': 0.15,
    'standing': 225,
    'running-treadmill': 0.1,
    'cycling-50W': 0.12,
    'cycling-100W': 0.12,
    'walking-left': 0.2,
    'walking-right': 0.2,
    'walking-impaired': 0.2,
    'walking': 0.25,
    'sitting': 400,
    'default': 0.5
}

In [6]:
# gait_sets_path = Path('/Users/adamol/Documents/Datasets/gait/processed')
gait_sets_path = Path('/home/lukasadamowicz/Documents/Datasets/processed')

datasets = [
    gait_sets_path / 'bluesky2',
    gait_sets_path / 'daliac',
    gait_sets_path / 'ltmm',
    gait_sets_path / 'usc-had'
]

X, Y, subjects, activities = load_datasets(
    datasets, 
    goal_fs=50.0, 
    acc_mag=True, 
    window_length=3.0, 
    window_step=steps
)

In [7]:
# make stair-climbing in the positive class
mask = (activities == 'stairs-ascending') | (activities == 'stairs-descending')
Y[mask] = 1

# Autocorrelation

In [8]:
lags = [1, 2, 5, 10, 12, 14, 15, 16, 18, 20, 25, 50, 100]

params = []
for l in lags:
    params.append({'lag': l, 'normalize': True})
        
test_feature_params(X, Y, Autocorrelation, params)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Sample Entropy

In [7]:
std = np.mean(np.std(X[Y > 0.5], axis=1))
std * 0.2

0.061182032079134356

In [10]:
M = [2, 3, 4, 5]
R = [0.05, 0.1, 0.25, 0.5, 0.75, 1.0]

params = []
for m in M:
    for r in R:
        params.append({'m': m, 'r': r})
        
test_feature_params(X, Y, SampleEntropy, params)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Permutation Entropy

In [11]:
ordr = [2, 3, 5, 8, 10]
dly = [1, 2, 3, 4, 5]

params = []
for o in ordr:
    for d in dly:
        params.append({'order': o, 'delay': d})
        
test_feature_params(X, Y, PermutationEntropy, params)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Range Count Percentage

In [13]:
rmin = [0, 0.1, 0.2, 0.3, 0.4, 0.5]
rmax = [0.6, 0.8, 1.0, 1.2, 1.4, 1.8, 2.0]

params = []
for rn in rmin:
    for rx in rmax:
        params.append({'range_min': rn, 'range_max': rx})
        
test_feature_params(X, Y, RangeCountPercentage, params)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Ratio Beyond R Sigma

In [14]:
R = [0.1, 0.2, 0.5, 1, 1.5, 2.0, 2.5, 3.0]

params = []
for r in R:
    params.append({'r': r})
        
test_feature_params(X, Y, RatioBeyondRSigma, params)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Dominant Frequency

In [15]:
low_cuts = [0.25, 0.5, 1.0, 1.5, 2.0, 2.5]
hi_cuts = [3.0, 3.5, 4.0, 5.0, 6.0]

params = []
for l in low_cuts:
    for h in hi_cuts:
        params.append({'low_cutoff': l, 'high_cutoff': h})
        
test_feature_params(X, Y, DominantFrequency, params)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Power Spectral Sum

In [16]:
low_cuts = [0.25, 0.5, 1.0, 1.5, 2.0, 2.5]
hi_cuts = [3.0, 3.5, 4.0, 5.0, 6.0]

params = []
for l in low_cuts:
    for h in hi_cuts:
        params.append({'low_cutoff': l, 'high_cutoff': h})
        
test_feature_params(X, Y, PowerSpectralSum, params)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Spectral Flatness

In [18]:
low_cuts = [0.0, 0.25, 0.5, 1.0, 1.5, 2.0, 2.5]
hi_cuts = [3.0, 3.5, 4.0, 5.0, 6.0, 7.0, 8.0]

params = []
for l in low_cuts:
    for h in hi_cuts:
        params.append({'low_cutoff': l, 'high_cutoff': h})
        
test_feature_params(X, Y, SpectralFlatness, params)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Spectral Entropy

In [19]:
low_cuts = [0.0, 0.25, 0.5, 1.0, 1.5, 2.0, 2.5]
hi_cuts = [3.0, 3.5, 4.0, 5.0, 6.0]

params = []
for l in low_cuts:
    for h in hi_cuts:
        params.append({'low_cutoff': l, 'high_cutoff': h})
        
test_feature_params(X, Y, SpectralEntropy, params)

<ipython-input-4-63541b70480b>:17: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  f, ax = plt.subplots(nrows=nrows, ncols=4, figsize=(10, 2.5 * nrows), sharex=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-4-63541b70480b>:38: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=(10, 5))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Detail Power Ratio

In [20]:
low_cuts = [1.0, 1.5]
hi_cuts = [4.0, 6.0]
waves = ['coif1', 'coif4', 'sym2', 'sym5', 'db1', 'db4', 'db7', 'bior1.3']

params = []
for wave in waves:
    for l in low_cuts:
        for h in hi_cuts:
            params.append({'wavelet': wave, 'freq_band': [l, h]})
        
test_feature_params(X, Y, DetailPowerRatio, params)

/home/lukasadamowicz/miniconda3/envs/pfymu/lib/python3.8/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 6 is too high: all coefficients will experience boundary effects.
  warnings.warn(
<ipython-input-4-63541b70480b>:17: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  f, ax = plt.subplots(nrows=nrows, ncols=4, figsize=(10, 2.5 * nrows), sharex=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-4-63541b70480b>:38: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=(10, 5))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …